# Tree Hazards in Oakland
#### by Wendy Chau

* I have decided to create a map of current tree hazards in Oakland.  This can be easily changed to different categories during the filtering process.  Using a CSV from the City of Oakland and plugging it into the SeeClickFix API I can perform different targeted queries.

In [1]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install gmaps

import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import requests
import json
import gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


## Parsing Oakland Service Request data

* As you can see below, SeeClickFix is the largest source of reports, with over 200k rows.  Note the 'REQUESTID' column as different number formats depending on the source, which unfortunately is NOT the same number as 'service_request-id' from the SeeClickFix API. Also note some sources (such as 'Phone or Email') can have no location data.
* To replicate these results with updated service reports, simply download the latest CSV from https://data.oaklandnet.com/Infrastructure/Service-requests-received-by-the-Oakland-Call-Cent/quth-gb8e and load it; simply by running the other cells you will start to compile the latest results.

In [2]:
file = "service_requests_032219.csv"
service_pd = pd.read_csv(file)
service_pd.head()

,REQUESTID,DATETIMEINIT,SOURCE,DESCRIPTION,REQCATEGORY,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT
0,812854,05/10/2018 09:24:33 AM,Phone,Tree - Request for Removal,TREES,"10305 FOOTHILL BLVD\nCA, CA\n(37.745114, -122....",OPEN,NaN,NaN,6083847.933,2097975.263,CCD7,32Y
1,21095,07/02/2009 09:33:03 AM,Phone or Email,Residential Recycling Service Issues,RECYCLING,"ZZ\nCA, CA",CLOSED,NaN,06/08/2010 09:24:48 AM,0.000,0.000,NaN,NaN
2,850653,11/01/2018 07:32:37 AM,SeeClickFix,"Illegal Dumping - debris, appliances, etc.",ILLDUMP,"2225 23RD AVE\nCA, CA\n(37.789934, -122.230485)",CLOSED,NaN,11/03/2018 06:14:27 PM,6061628.445,2114717.454,CCD2,18Y
3,21090,07/01/2009 10:24:19 AM,Phone or Email,Yard Trimmings - Food Pail Request,RECYCLING,"532 BODEN WY\nCA, CA\n(37.807708, -122.248339)",CLOSED,NaN,07/27/2009 11:34:09 AM,6056502.139,2121252.658,CCD3,15X
4,21091,07/01/2009 11:14:09 AM,Phone or Email,City Services - Missed Recycling,RECYCLING,"699 BELLEVUE AV\nCA, CA\n(37.809762, -122.259388)",CLOSED,NaN,07/03/2009 11:10:08 AM,6053405.037,2122019.368,CCD3,14X


In [3]:
# note how many results SeeClickFix has compared to all other methods

source = pd.DataFrame(service_pd.groupby("SOURCE")['REQUESTID'].nunique())
source = source.rename(columns = {"REQUESTID":"COUNT"})
source

,COUNT
SOURCE,
City Attorney,189
Email,33219
Other,3543
Phone,114496
Phone or Email,140782
SeeClickFix,200545
Voicemail,8036
Website,77143


In [4]:
# isolate SeeClickFix

seeclickfix_pd = pd.DataFrame(service_pd.loc[service_pd["SOURCE"] == "SeeClickFix", :])
seeclickfix_pd = seeclickfix_pd.reset_index(drop=True)
seeclickfix_pd.head()

,REQUESTID,DATETIMEINIT,SOURCE,DESCRIPTION,REQCATEGORY,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT
0,850653,11/01/2018 07:32:37 AM,SeeClickFix,"Illegal Dumping - debris, appliances, etc.",ILLDUMP,"2225 23RD AVE\nCA, CA\n(37.789934, -122.230485)",CLOSED,NaN,11/03/2018 06:14:27 PM,6061628.445,2114717.454,CCD2,18Y
1,633815,02/09/2016 04:20:16 PM,SeeClickFix,Graffiti on OTHER,OTHER,"8TH ST &amp; BROADWAY\nCA, CA\n(33.100174, -11...",REFERRED,"USPS,",NaN,6049216.500,2118687.750,CCD3,03X
2,812811,05/10/2018 07:45:55 AM,SeeClickFix,Illegal Dumping � mattress/boxspring,ILLDUMP,"1072 57TH ST\nCA, CA\n(38.562959, -121.434347)",CLOSED,NaN,05/14/2018 12:53:30 PM,6047218.212,2133462.098,CCD1,10X
3,812812,05/10/2018 07:48:44 AM,SeeClickFix,"Illegal Dumping - debris, appliances, etc.",ILLDUMP,"5714 SAN PABLO AVE\nCA, CA\n(37.840517, -122.2...",CLOSED,NaN,05/14/2018 12:54:13 PM,6046951.224,2133412.247,CCD1,10X
4,850702,11/01/2018 09:16:13 AM,SeeClickFix,"Graffiti on Street, Street Light, Traffic Signal,",GRAFFITI,"3504 RICHMOND BLVD\nCA, CA\n(37.821472, -122.2...",CLOSED,NaN,11/02/2018 09:55:53 AM,6054270.967,2126262.088,CCD3,09X


In [5]:
status = pd.DataFrame(seeclickfix_pd.groupby("STATUS")['REQUESTID'].nunique())
status = status.rename(columns = {"REQUESTID":"COUNT"})
status

,COUNT
STATUS,
CANCEL,38469
CLOSED,108679
Cancel,1
EVALUATED - NO FURTHER ACTION,30
GONE ON ARRIVAL,33
OPEN,17127
PENDING,6341
REFERRED,25001
UNFUNDED,1160


## Additional filtering

* I want to isolate, for example, reports relating to trees that are still ongoing.  I search for the word 'Tree' in the 'DESCRIPTION" column, which does NOT pull up "Streets - Potholes/Depression"
* Unfortunately since Oakland doesn't retain the user-input description which can contain a lot flavorful commentary; for example one request the user wrote "In front of 906 Washington st. There  is a tree grate that has been an trip hazard for quite sometime, please assist with resolving this issue."

In [6]:
# filter by open/pending/unfunded/waiting_on_customer reports

active_pd = pd.DataFrame(seeclickfix_pd.loc[(seeclickfix_pd["STATUS"] == "OPEN") | (
    seeclickfix_pd["STATUS"] == "PENDING") | (seeclickfix_pd["STATUS"] == "UNFUNDED") | (
    seeclickfix_pd["STATUS"] == "WAITING ON CUSTOMER"), :])

active_pd = active_pd.reset_index(drop=True)
active_pd.head()

,REQUESTID,DATETIMEINIT,SOURCE,DESCRIPTION,REQCATEGORY,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT
0,813027,05/11/2018 03:49:23 AM,SeeClickFix,Streets - Potholes/Depression,STREETSW,"2757 PARKER AV\nCA, CA\n(37.765585, -122.168617)",OPEN,NaN,NaN,6079316.043,2105480.224,CCD6,30Y
1,850862,11/01/2018 05:44:39 PM,SeeClickFix,Hazardous Materials,FIRE,"724 GLENDOME CIR\nCA, CA\n(37.812902, -122.21949)",OPEN,NaN,NaN,6065063.461,2123012.437,CCD5,16Y
2,817275,06/02/2018 12:24:49 PM,SeeClickFix,Inspections - Utility-Related Infrastructure,ROW_INSPECTORS,"5001 DAISY ST\nCA, CA\n(37.784656, -122.18241)",OPEN,NaN,NaN,6075245.836,2112459.846,CCD4,25X
3,756159,08/29/2017 10:03:41 AM,SeeClickFix,Watershed - Other/Complex,WATERSHED,"12530 BROOKPARK RD\nCA, CA\n(37.795233, -122.1...",OPEN,NaN,NaN,6082602.910,2116226.540,CCD6,25Y
4,813246,05/11/2018 09:50:26 PM,SeeClickFix,Streets - Potholes/Depression,STREETSW,"272 BEAUFOREST DRIVE\nCA, CA\n(37.841914, -122...",OPEN,NaN,NaN,6068105.048,2133453.849,CCD4,13Y


In [7]:
# filter by tree issues

dangerous_trees = active_pd[active_pd["DESCRIPTION"].str.match('Tree')]
dangerous_trees = dangerous_trees.reset_index(drop=True)
dangerous_trees.head()

,REQUESTID,DATETIMEINIT,SOURCE,DESCRIPTION,REQCATEGORY,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT
0,771465,11/04/2017 03:30:20 PM,SeeClickFix,Tree - Restake,TREES,"522 ZORAH ST\nCA, CA\n(37.803187, -122.242538)",UNFUNDED,NaN,NaN,6058323.830,2119572.120,CCD2,15X
1,771947,11/06/2017 02:08:59 PM,SeeClickFix,Tree - Pruning,TREES,"262 2ND ST\nCA, CA\n(37.78582, -122.397041)",UNFUNDED,NaN,NaN,6049700.090,2116565.210,CCD3,01X
2,814568,05/18/2018 03:20:47 PM,SeeClickFix,Tree - Pruning,TREES,"MEDIAN SKYLINE - STABLES\nCA, CA",OPEN,NaN,NaN,6087231.445,2111943.565,CCD6,25Y
3,852431,11/08/2018 02:29:43 PM,SeeClickFix,Tree - Pruning,TREES,"GARFIELD PARK\nCA, CA",OPEN,NaN,NaN,6060557.396,2113203.099,CCD2,18X
4,766327,10/14/2017 02:21:30 PM,SeeClickFix,Tree - Pruning,TREES,"603 LANCASTER ST\nCA, CA\n(37.77217, -122.229708)",OPEN,NaN,NaN,6061718.520,2108205.640,CCD5,20X


In [8]:
len(dangerous_trees)

729

In [9]:
# alternative - not used - filter by tree issues using a different column, note the results are very close

dangerous_trees_alt = active_pd[active_pd["REQCATEGORY"].str.match('TREES')]
dangerous_trees_alt = dangerous_trees_alt.reset_index(drop=True)
dangerous_trees_alt.head()

,REQUESTID,DATETIMEINIT,SOURCE,DESCRIPTION,REQCATEGORY,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT
0,771465,11/04/2017 03:30:20 PM,SeeClickFix,Tree - Restake,TREES,"522 ZORAH ST\nCA, CA\n(37.803187, -122.242538)",UNFUNDED,NaN,NaN,6058323.830,2119572.120,CCD2,15X
1,771947,11/06/2017 02:08:59 PM,SeeClickFix,Tree - Pruning,TREES,"262 2ND ST\nCA, CA\n(37.78582, -122.397041)",UNFUNDED,NaN,NaN,6049700.090,2116565.210,CCD3,01X
2,814568,05/18/2018 03:20:47 PM,SeeClickFix,Tree - Pruning,TREES,"MEDIAN SKYLINE - STABLES\nCA, CA",OPEN,NaN,NaN,6087231.445,2111943.565,CCD6,25Y
3,852431,11/08/2018 02:29:43 PM,SeeClickFix,Tree - Pruning,TREES,"GARFIELD PARK\nCA, CA",OPEN,NaN,NaN,6060557.396,2113203.099,CCD2,18X
4,766327,10/14/2017 02:21:30 PM,SeeClickFix,Tree - Pruning,TREES,"603 LANCASTER ST\nCA, CA\n(37.77217, -122.229708)",OPEN,NaN,NaN,6061718.520,2108205.640,CCD5,20X


In [10]:
len(dangerous_trees_alt)

746

In [11]:
tree_description = pd.DataFrame(dangerous_trees.groupby("DESCRIPTION")['REQUESTID'].nunique())
tree_description = tree_description.rename(columns = {"REQUESTID":"COUNT"})
tree_description

,COUNT
DESCRIPTION,
Tree - Broken/Hanging Limb,4
"Tree - Diseased, Dead, Dying, Leaning, or Split",136
Tree - Down,29
Tree - Illegal Tree Removal,23
Tree - Limb Down,11
Tree - Other/Complex,34
Tree - Permit - Waiver,1
Tree - Pruning,432
Tree - Request for Removal,8


## Converting coordinates

* Oakland stores the location data in 'NAD 1983 StatePlane California III FIPS 0403 Feet' format.  There is a free API that converts this into latitude & longititude (plus the other way around - if desired). URL: http://epsg.io/ using JSON instructions found in their repository https://github.com/klokantech/epsg.io

In [ ]:
# convert location data to latitude/longitude
# NOTE: I could also grab latitude/longitude from 'REQADDRESS' but it doesn't exist in all rows

latitude = []
longitude = []


print("BEGINNING COORDINATE CONVERSION")
print("------------------------------")

for i in range(0,len(dangerous_trees)):
    print(f"CONVERTING DATA FROM ROW #{i}")
    
    SRX = dangerous_trees["SRX"][i]
    SRY = dangerous_trees["SRY"][i]

    url = "http://epsg.io/trans?x={0}&y={1}&s_srs=102643&t_srs=4326".format(SRX, SRY)
    data = requests.get(url).json()
    
    try:
        longitude.append(data['x'])
    except (KeyError, IndexError):
        print(f"- missing longitute data for row#{i}")
        longitude.append(None)
        
    try:
        latitude.append(data['y'])
    except (KeyError, IndexError):
        print(f"- missing latitude data for row#{i}")
        longitude.append(None)
    
print("------------------------------")    
print("DATA RETRIEVAL COMPLETED")

In [ ]:
# export results to avoid running query twice

report_coordinates = pd.DataFrame({"LATITUDE": latitude, "LONGITUDE": longitude})
report_coordinates.head()

report_coordinates.to_csv("report_coordinates_copy.csv", index=True, header=True)

In [12]:
# import coordinates CSV

coord_file = "report_coordinates.csv"
coordinates_pd = pd.read_csv(coord_file)
coordinates_pd = coordinates_pd.drop(["Unnamed: 0"], axis=1)
coordinates_pd.head()

,LATITUDE,LONGITUDE
0,37.803094,-122.242301
1,37.794393,-122.271948
2,37.783582,-122.141797
3,37.785719,-122.234163
4,37.772056,-122.229826


In [13]:
dangerous_trees["LATITUDE"] = coordinates_pd["LATITUDE"]
dangerous_trees["LONGITUDE"] = coordinates_pd["LONGITUDE"]
dangerous_trees.head(10)

,REQUESTID,DATETIMEINIT,SOURCE,DESCRIPTION,REQCATEGORY,REQADDRESS,STATUS,REFERREDTO,DATETIMECLOSED,SRX,SRY,COUNCILDISTRICT,BEAT,LATITUDE,LONGITUDE
0,771465,11/04/2017 03:30:20 PM,SeeClickFix,Tree - Restake,TREES,"522 ZORAH ST\nCA, CA\n(37.803187, -122.242538)",UNFUNDED,NaN,NaN,6058323.830,2119572.120,CCD2,15X,37.803094,-122.242301
1,771947,11/06/2017 02:08:59 PM,SeeClickFix,Tree - Pruning,TREES,"262 2ND ST\nCA, CA\n(37.78582, -122.397041)",UNFUNDED,NaN,NaN,6049700.090,2116565.210,CCD3,01X,37.794393,-122.271948
2,814568,05/18/2018 03:20:47 PM,SeeClickFix,Tree - Pruning,TREES,"MEDIAN SKYLINE - STABLES\nCA, CA",OPEN,NaN,NaN,6087231.445,2111943.565,CCD6,25Y,37.783582,-122.141797
3,852431,11/08/2018 02:29:43 PM,SeeClickFix,Tree - Pruning,TREES,"GARFIELD PARK\nCA, CA",OPEN,NaN,NaN,6060557.396,2113203.099,CCD2,18X,37.785719,-122.234163
4,766327,10/14/2017 02:21:30 PM,SeeClickFix,Tree - Pruning,TREES,"603 LANCASTER ST\nCA, CA\n(37.77217, -122.229708)",OPEN,NaN,NaN,6061718.520,2108205.640,CCD5,20X,37.772056,-122.229826
5,855699,11/26/2018 05:18:44 PM,SeeClickFix,Tree - Pruning,TREES,"2704 E 22ND ST\nCA, CA\n(37.78647, -122.22587)",OPEN,NaN,NaN,6062962.542,2113447.408,CCD5,21X,37.786512,-122.225857
6,457567,04/26/2013 08:21:25 AM,SeeClickFix,Tree - Pruning,TREES,"69TH AV &amp; OUTLOOK AV\nCA, CA\n(37.772223, ...",OPEN,NaN,NaN,6078126.500,2107967.500,CCD6,29X,37.772221,-122.173052
7,522357,06/08/2014 12:30:16 AM,SeeClickFix,Tree - Pruning,TREES,"PLEASANT VALLEY AV &amp; MONTGOMERY ST\nCA, CA...",OPEN,NaN,NaN,6056613.000,2129962.250,CCD1,09X,37.831536,-122.248894
8,531128,07/25/2014 01:10:15 PM,SeeClickFix,"Tree - Diseased, Dead, Dying, Leaning, or Split",TREES,"KELLER AV &amp; CAMPUS DR\nCA, CA\n(37.77315, ...",OPEN,NaN,NaN,6086801.510,2108001.240,CCD7,35Y,37.772736,-122.143045
9,532637,08/02/2014 04:54:46 PM,SeeClickFix,Tree - Pruning,TREES,"PROSPECT AV &amp; KENWYN RD\nCA, CA\n(37.80617...",OPEN,NaN,NaN,6057720.000,2120689.750,CCD2,15X,37.806132,-122.244463


In [76]:
# trying to find relevant service codes in SeeClickFix - this is brute force based on info in app

code_number = []
service_name = []

print("BEGINNING IMAGE RETRIEVAL")
print("------------------------------")
    
for i in range(1120,1140):
    print(f"TRYING SERVICE CODE {i}")

    url = "https://seeclickfix.com/open311/v2/requests.json?lat=37.8044&long=-122.2711&service_code={0}".\
        format(i)
    data = requests.get(url).json()
    
    try:
        code_number.append(data[0]['service_code'])
        service_name.append(data[0]['service_name'])
    except (KeyError, IndexError, TypeError):
        print(f"- no such service code: {i}")
        code_number.append(i)
        service_name.append([None])
        
print("------------------------------")    
print("DATA RETRIEVAL COMPLETED")

codes_pd = pd.DataFrame({"CODE_NUMBER": code_number, "SERVICE_NAME": service_name})
codes_pd

BEGINNING IMAGE RETRIEVAL
------------------------------
TRYING SERVICE CODE 1120
- no such service code: 1120
TRYING SERVICE CODE 1121
- no such service code: 1121
TRYING SERVICE CODE 1122
TRYING SERVICE CODE 1123
TRYING SERVICE CODE 1124
TRYING SERVICE CODE 1125
TRYING SERVICE CODE 1126
TRYING SERVICE CODE 1127
TRYING SERVICE CODE 1128
TRYING SERVICE CODE 1129
TRYING SERVICE CODE 1130
TRYING SERVICE CODE 1131
TRYING SERVICE CODE 1132
TRYING SERVICE CODE 1133
TRYING SERVICE CODE 1134
- no such service code: 1134
TRYING SERVICE CODE 1135
- no such service code: 1135
TRYING SERVICE CODE 1136
- no such service code: 1136
TRYING SERVICE CODE 1137
- no such service code: 1137
TRYING SERVICE CODE 1138
- no such service code: 1138
TRYING SERVICE CODE 1139
- no such service code: 1139
------------------------------
DATA RETRIEVAL COMPLETED


,CODE_NUMBER,SERVICE_NAME
0,1120,[None]
1,1121,[None]
2,1122,Graffiti
3,1123,Illegal Dumping
4,1124,Street Light (Not Traffic Signal)
5,1125,Street/Traffic Issue (Other than Pothole)
6,1126,Sidewalk
7,1127,Park Maintenance
8,1128,Other: Drainage / Flooding
9,1129,Other: Sewers


## Retrieving from SeeClickFix API

* I am pulling data from several search results from the coordinates under the service code for trees, which should find the most relevant result / the closest match on some listings.  I am grabbing images, commentary from the user who submitted the report, and also check the Oakland Service dates against the SeeClickFix dates to determine if I found a match. URL: https://seeclickfix.com/open311/v2/docs using instructions on http://dev.seeclickfix.com/ 

In [15]:
images0 = []
images1 = []
images2 = []
images3 = []
images4 = []
images5 = []

descriptions0 = []
descriptions1 = []
descriptions2 = []
descriptions3 = []
descriptions4 = []
descriptions5 = []

request_dates0 = []
request_dates1 = []
request_dates2 = []
request_dates3 = []
request_dates4 = []
request_dates5 = []

print("BEGINNING DATA RETRIEVAL")
print("------------------------------")
    
for i in range(0,len(dangerous_trees)):
    print(f"RETRIEVING DATA FOR INCIDENT #{i}")
    
    
    LAT = dangerous_trees["LATITUDE"][i]
    LNG = dangerous_trees["LONGITUDE"][i]

    url = "https://seeclickfix.com/open311/v2/requests.json?lat={0}&long={1}&service_code=1132".\
        format(LAT, LNG)
    data = requests.get(url).json()
    
    print("Retrieving first result")
    try:
        images0.append(data[0]['media_url'])
    except (KeyError, IndexError, TypeError):
        print(f"row#{i} - no image")
        images0.append(None)
    try:
        descriptions0.append(data[0]['description'])
    except (KeyError, IndexError, TypeError):
        print(f"row#{i} - no description")
        descriptions0.append(None)
    try:
        request_dates0.append(data[0]['requested_datetime'])
    except (KeyError, IndexError, TypeError):
        print(f"row#{i} - no timestamp")
        request_dates0.append(None)

# I am searching further down the results
    
    print("Retrieving second result")
    try:
        images1.append(data[1]['media_url'])
    except (KeyError, IndexError, TypeError):
        print(f"row#{i} - no image")
        images1.append(None) 
    try:
        descriptions1.append(data[1]['description'])
    except (KeyError, IndexError, TypeError):
        print(f"row#{i} - no description")
        descriptions1.append(None)
    try:
        request_dates1.append(data[1]['requested_datetime'])
    except (KeyError, IndexError, TypeError):
        print(f"row#{i} - no image")
        request_dates1.append(None)
    
    print("Retrieving third result")
    try:
        images2.append(data[2]['media_url'])
    except (KeyError, IndexError, TypeError):
        print(f"row#{i} - no image")
        images2.append(None)
    try:
        descriptions2.append(data[2]['description'])
    except (KeyError, IndexError, TypeError):
        print(f"row#{i} - no description")
        descriptions2.append(None)
    try:
        request_dates2.append(data[2]['requested_datetime'])
    except (KeyError, IndexError, TypeError):
        print(f"row#{i} - no timestamp")
        request_dates2.append(None)
      
    print("Retrieving fourth result")
    try:
        images3.append(data[3]['media_url'])
    except (KeyError, IndexError, TypeError):
        print(f"row#{i} - no image")
        images3.append(None) 
    try:
        descriptions3.append(data[3]['description'])
    except (KeyError, IndexError, TypeError):
        print(f"row#{i} - no description")
        descriptions3.append(None)
    try:
        request_dates3.append(data[3]['requested_datetime'])
    except (KeyError, IndexError, TypeError):
        print(f"row#{i} - no timestamp")
        request_dates3.append(None)
        
    print("Retrieving fifth result")
    try:
        images4.append(data[4]['media_url'])
    except (KeyError, IndexError, TypeError):
        print(f"row#{i} - no image")
        images4.append(None) 
    try:
        descriptions4.append(data[4]['description'])
    except (KeyError, IndexError, TypeError):
        print(f"row#{i} - no description")
        descriptions4.append(None)
    try:
        request_dates4.append(data[4]['requested_datetime'])
    except (KeyError, IndexError, TypeError):
        print(f"row#{i} - no timestamp")
        request_dates4.append(None)
        
    print("Retrieving sixth result")
    try:
        images5.append(data[5]['media_url'])
    except (KeyError, IndexError, TypeError):
        print(f"row#{i} - no image")
        images5.append(None) 
    try:
        descriptions5.append(data[5]['description'])
    except (KeyError, IndexError, TypeError):
        print(f"row#{i} - no description")
        descriptions5.append(None)
    try:
        request_dates5.append(data[5]['requested_datetime'])
    except (KeyError, IndexError, TypeError):
        print(f"row#{i} - no timestamp")
        request_dates5.append(None)
        
        
print("------------------------------")    
print("DATA RETRIEVAL COMPLETED")

BEGINNING DATA RETRIEVAL
------------------------------
RETRIEVING DATA FOR INCIDENT #0
Retrieving first result
Retrieving second result
Retrieving third result
row#0 - no image
row#0 - no description
row#0 - no timestamp
Retrieving fourth result
row#0 - no image
row#0 - no description
row#0 - no timestamp
Retrieving fifth result
row#0 - no image
row#0 - no description
row#0 - no timestamp
Retrieving sixth result
row#0 - no image
row#0 - no description
row#0 - no timestamp
RETRIEVING DATA FOR INCIDENT #1
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #2
Retrieving first result
Retrieving second result
row#2 - no image
row#2 - no description
row#2 - no image
Retrieving third result
row#2 - no image
row#2 - no description
row#2 - no timestamp
Retrieving fourth result
row#2 - no image
row#2 - no description
row#2 - no timestamp
Retrieving fifth result
row#2 - no 

Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #33
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #34
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #35
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
row#35 - no image
row#35 - no description
row#35 - no timestamp
RETRIEVING DATA FOR INCIDENT #36
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #37
Retrieving first result
Retrieving second

Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #68
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #69
Retrieving first result
row#69 - no image
row#69 - no description
row#69 - no timestamp
Retrieving second result
row#69 - no image
row#69 - no description
row#69 - no image
Retrieving third result
row#69 - no image
row#69 - no description
row#69 - no timestamp
Retrieving fourth result
row#69 - no image
row#69 - no description
row#69 - no timestamp
Retrieving fifth result
row#69 - no image
row#69 - no description
row#69 - no timestamp
Retrieving sixth result
row#69 - no image
row#69 - no description
row#69 - no timestamp
RETRIEVING DATA FOR INCIDENT #70
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving

Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
row#99 - no image
row#99 - no description
row#99 - no timestamp
Retrieving fifth result
row#99 - no image
row#99 - no description
row#99 - no timestamp
Retrieving sixth result
row#99 - no image
row#99 - no description
row#99 - no timestamp
RETRIEVING DATA FOR INCIDENT #100
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #101
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #102
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #103
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving four

Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #129
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #130
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #131
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
row#131 - no image
row#131 - no description
row#131 - no timestamp
Retrieving fifth result
row#131 - no image
row#131 - no description
row#131 - no timestamp
Retrieving sixth result
row#131 - no image
row#131 - no description
row#131 - no timestamp
RETRIEVING DATA FOR INCIDENT #132
Retrieving first result
Retrieving second result
Retrieving third result
Retrie

...

Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #584
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #585
Retrieving first result
Retrieving second result
row#585 - no image
row#585 - no description
row#585 - no image
Retrieving third result
row#585 - no image
row#585 - no description
row#585 - no timestamp
Retrieving fourth result
row#585 - no image
row#585 - no description
row#585 - no timestamp
Retrieving fifth result
row#585 - no image
row#585 - no description
row#585 - no timestamp
Retrieving sixth result
row#585 - no image
row#585 - no description
row#585 - no timestamp
RETRIEVING DATA FOR INCIDENT #586
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
row#586 - no image
row#586 - no

Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
row#616 - no image
row#616 - no description
row#616 - no timestamp
Retrieving fifth result
row#616 - no image
row#616 - no description
row#616 - no timestamp
Retrieving sixth result
row#616 - no image
row#616 - no description
row#616 - no timestamp
RETRIEVING DATA FOR INCIDENT #617
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #618
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #619
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
row#619 - no image
row#619 - no description
row#619 - no timestamp
RETRIEVING DATA FOR INCIDENT #620
Retrieving f

Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #649
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
row#649 - no image
row#649 - no description
row#649 - no timestamp
Retrieving fifth result
row#649 - no image
row#649 - no description
row#649 - no timestamp
Retrieving sixth result
row#649 - no image
row#649 - no description
row#649 - no timestamp
RETRIEVING DATA FOR INCIDENT #650
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #651
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #652
Retrieving first result
Retrieving second result
Retrieving third result
Retrie

Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #681
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #682
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #683
Retrieving first result
Retrieving second result
row#683 - no image
row#683 - no description
row#683 - no image
Retrieving third result
row#683 - no image
row#683 - no description
row#683 - no timestamp
Retrieving fourth result
row#683 - no image
row#683 - no description
row#683 - no timestamp
Retrieving fifth result
row#683 - no image
row#683 - no description
row#683 - no timestamp
Retrieving sixth result
row#683 - no image
row#683 - no description
row#68

Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #716
Retrieving first result
Retrieving second result
Retrieving third result
row#716 - no image
row#716 - no description
row#716 - no timestamp
Retrieving fourth result
row#716 - no image
row#716 - no description
row#716 - no timestamp
Retrieving fifth result
row#716 - no image
row#716 - no description
row#716 - no timestamp
Retrieving sixth result
row#716 - no image
row#716 - no description
row#716 - no timestamp
RETRIEVING DATA FOR INCIDENT #717
Retrieving first result
Retrieving second result
Retrieving third result
Retrieving fourth result
Retrieving fifth result
Retrieving sixth result
RETRIEVING DATA FOR INCIDENT #718
Retrieving first result
Retrieving second result
Retrieving third result
row#718 - no image
row#718 - no description
row#718 - no timestamp
Retrieving fourth result
row#718 - no image
row#718 

In [17]:
# organizing the results

images_pd = pd.DataFrame({"IMAGE_URL_0": images0, "DESCRIPTION_0": descriptions0, "SEECLICKFIX_DATE_0": request_dates0,
    "IMAGE_URL_1": images1, "DESCRIPTION_1": descriptions1, "SEECLICKFIX_DATE_1": request_dates1,
    "IMAGE_URL_2": images2, "DESCRIPTION_2": descriptions2, "SEECLICKFIX_DATE_2": request_dates2,
    "IMAGE_URL_3": images3, "DESCRIPTION_3": descriptions3, "SEECLICKFIX_DATE_3": request_dates3,
    "IMAGE_URL_4": images4, "DESCRIPTION_4": descriptions4, "SEECLICKFIX_DATE_4": request_dates4,
    "IMAGE_URL_5": images5, "DESCRIPTION_5": descriptions5, "SEECLICKFIX_DATE_5": request_dates5})
images_pd["OAK_311_DATE"] = dangerous_trees["DATETIMEINIT"]
images_pd["LATITUDE"] = dangerous_trees["LATITUDE"]
images_pd["LONGITUDE"] = dangerous_trees["LONGITUDE"]

        
images_pd.to_csv("image_location_data.csv", index=True, header=True)
images_pd.head(10)

,IMAGE_URL_0,DESCRIPTION_0,SEECLICKFIX_DATE_0,IMAGE_URL_1,DESCRIPTION_1,SEECLICKFIX_DATE_1,IMAGE_URL_2,DESCRIPTION_2,SEECLICKFIX_DATE_2,IMAGE_URL_3,...,SEECLICKFIX_DATE_3,IMAGE_URL_4,DESCRIPTION_4,SEECLICKFIX_DATE_4,IMAGE_URL_5,DESCRIPTION_5,SEECLICKFIX_DATE_5,OAK_311_DATE,LATITUDE,LONGITUDE
0,https://seeclickfix.com/files/issue_images/001...,Overgrown and blocking side walk.,2013-07-17T22:51:11-04:00,None,"Many years ago, the City of Oakland staff plan...",2016-01-29T12:34:56-05:00,None,None,None,None,...,None,None,None,None,None,None,None,11/04/2017 03:30:20 PM,37.803094,-122.242301
1,https://seeclickfix.com/files/issue_images/001...,Trees on public sidewalk need to be trimmed. ...,2013-09-18T14:55:11-04:00,https://seeclickfix.com/files/issue_images/001...,Treedown at fourth and Alice,2014-03-20T10:10:48-04:00,https://seeclickfix.com/files/issue_images/003...,Huge branch fallen down in street,2015-09-14T17:11:33-04:00,https://seeclickfix.com/files/issue_images/004...,...,2015-11-11T15:55:10-05:00,https://seeclickfix.com/files/issue_images/003...,Huge branch on sidewalk,2015-09-04T12:53:07-04:00,https://seeclickfix.com/files/issue_images/004...,Branch broken,2015-11-12T19:21:18-05:00,11/06/2017 02:08:59 PM,37.794393,-122.271948
2,None,Branches cut by city during Dec storms have be...,2015-01-30T17:04:12-05:00,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,05/18/2018 03:20:47 PM,37.783582,-122.141797
3,https://seeclickfix.com/files/issue_images/001...,This tree needs to be replaced!,2013-09-08T12:15:58-04:00,https://seeclickfix.com/files/issue_images/002...,The residence located at 2247 E. 15th Street h...,2015-02-08T19:01:39-05:00,None,None,None,None,...,None,None,None,None,None,None,None,11/08/2018 02:29:43 PM,37.785719,-122.234163
4,https://seeclickfix.com/files/issue_images/003...,need some maintainance.,2015-04-08T20:44:23-04:00,None,hazardous tree at the residence: 3232 Elmwood...,2016-11-04T12:32:16-04:00,https://seeclickfix.com/files/issue_images/002...,Large fallen tree branch (on my car - I was ab...,2014-08-13T23:18:33-04:00,https://seeclickfix.com/files/issue_images/002...,...,2014-08-13T21:46:17-04:00,None,The palm tree in front of 614 Lancaster is ful...,2016-12-22T17:28:00-05:00,None,None,None,10/14/2017 02:21:30 PM,37.772056,-122.229826
5,https://seeclickfix.com/files/issue_images/001...,Please come trim these trees they are too tall...,2013-11-23T10:57:46-05:00,https://seeclickfix.com/files/issue_images/001...,Tree uprooted looks like it was hit by a car. ...,2014-02-23T14:03:37-05:00,https://seeclickfix.com/files/issue_images/001...,Please come trim the trees in the creek tree f...,2013-11-23T10:59:48-05:00,None,...,2014-10-03T18:35:44-04:00,https://seeclickfix.com/files/issue_images/003...,County eucalyptus trees overhanging private pr...,2015-06-22T14:01:52-04:00,https://seeclickfix.com/files/issue_images/008...,The tree canopy is too low. It has attracted ...,2017-09-25T22:12:12-04:00,11/26/2018 05:18:44 PM,37.786512,-122.225857
6,None,"On MacArthur in the westbound direction, a tre...",2015-04-08T03:27:43-04:00,https://seeclickfix.com/files/issue_images/002...,No one has trimmed dis @#$% of tree yet! No on...,2014-11-05T12:21:40-05:00,https://seeclickfix.com/files/issue_images/002...,Untrimmed bottom foliage blocking driver's vi...,2014-07-08T02:16:59-04:00,https://seeclickfix.com/files/issue_images/008...,...,2017-06-22T13:11:43-04:00,None,None,None,None,None,None,04/26/2013 08:21:25 AM,37.772221,-122.173052
7,https://seeclickfix.com/files/issue_images/002...,The landscaping along the East side of Pleasan...,2014-09-02T19:03:44-04:00,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,06/08/2014 12:30:16 AM,37.831536,-122.248894
8,https://seeclickfix.com/files/issue_images/003...,downed tree from car crash,2015-03-20T18:57:38-04:00,None,Tree down blocking rt lane east bound Keller ave,2014-05-02T19:10:57-04:00,None,downed trees in median strip\r\nthis has 

## Parsing data retrieved

* To find if my query found the listing rather than something similar, I will split the month, year, and day (plus or minus one day as a workaround because SeeClickFix stores the info in ISO 8601 Notation) and compare with Oakland's database.  I will use the results if the date matches.

In [103]:
# cleaning the date formats from Oakland's dataset

oak_date_clean = pd.DataFrame(images_pd["OAK_311_DATE"].\
    str.split(' ', expand=True)).rename(columns = lambda x: "part_"+str(x))

oak_date_clean = oak_date_clean.rename(index=int, columns={"part_0": "OAK_311_DATE"}).\
    drop(["part_1"], axis=1).drop(["part_2"], axis=1)

oak_date_split = pd.DataFrame(oak_date_clean["OAK_311_DATE"].\
    str.split('/', expand=True)).rename(columns = lambda x: "date_"+str(x))

oak_date_split = oak_date_split.\
    rename(index=int, columns={"date_0": "OAK_MM", "date_1": "OAK_DD", "date_2": "OAK_YYYY"})

oak_date_split.head()

,OAK_MM,OAK_DD,OAK_YYYY
0,11,04,2017
1,11,06,2017
2,05,18,2018
3,11,08,2018
4,10,14,2017


In [108]:
# cleaning the date formats from SeeClickFix

date_clean0 = pd.DataFrame(images_pd["SEECLICKFIX_DATE_0"].\
    str.split('T', expand=True)).rename(columns = lambda x: "part_"+str(x))
date_clean0 = date_clean0.rename(index=int, columns={"part_0": "SEECLICKFIX_DATE_0"}).\
    drop(["part_1"], axis=1)
date_0_split = pd.DataFrame(date_clean0["SEECLICKFIX_DATE_0"].\
    str.split('-', expand=True)).rename(columns = lambda x: "date_"+str(x))
date_0_split = date_0_split.\
    rename(index=int, columns={"date_0": "YYYY", "date_1": "MM", "date_2": "DD"})

date_clean1 = pd.DataFrame(images_pd["SEECLICKFIX_DATE_1"].\
    str.split('T', expand=True)).rename(columns = lambda x: "part_"+str(x))
date_clean1 = date_clean1.rename(index=int, columns={"part_0": "SEECLICKFIX_DATE_1"}).\
    drop(["part_1"], axis=1)
date_1_split = pd.DataFrame(date_clean1["SEECLICKFIX_DATE_1"].\
    str.split('-', expand=True)).rename(columns = lambda x: "date_"+str(x))
date_1_split = date_1_split.\
    rename(index=int, columns={"date_0": "YYYY", "date_1": "MM", "date_2": "DD"})

date_clean2 = pd.DataFrame(images_pd["SEECLICKFIX_DATE_2"].\
    str.split('T', expand=True)).rename(columns = lambda x: "part_"+str(x))
date_clean2 = date_clean2.rename(index=int, columns={"part_0": "SEECLICKFIX_DATE_2"}).\
    drop(["part_1"], axis=1)
date_2_split = pd.DataFrame(date_clean2["SEECLICKFIX_DATE_2"].\
    str.split('-', expand=True)).rename(columns = lambda x: "date_"+str(x))
date_2_split = date_2_split.\
    rename(index=int, columns={"date_0": "YYYY", "date_1": "MM", "date_2": "DD"})

date_clean3 = pd.DataFrame(images_pd["SEECLICKFIX_DATE_3"].\
    str.split('T', expand=True)).rename(columns = lambda x: "part_"+str(x))
date_clean3 = date_clean3.rename(index=int, columns={"part_0": "SEECLICKFIX_DATE_3"}).\
    drop(["part_1"], axis=1)
date_3_split = pd.DataFrame(date_clean3["SEECLICKFIX_DATE_3"].\
    str.split('-', expand=True)).rename(columns = lambda x: "date_"+str(x))
date_3_split = date_3_split.\
    rename(index=int, columns={"date_0": "YYYY", "date_1": "MM", "date_2": "DD"})

date_clean4 = pd.DataFrame(images_pd["SEECLICKFIX_DATE_4"].\
    str.split('T', expand=True)).rename(columns = lambda x: "part_"+str(x))
date_clean4 = date_clean4.rename(index=int, columns={"part_0": "SEECLICKFIX_DATE_4"}).\
    drop(["part_1"], axis=1)
date_4_split = pd.DataFrame(date_clean4["SEECLICKFIX_DATE_4"].\
    str.split('-', expand=True)).rename(columns = lambda x: "date_"+str(x))
date_4_split = date_4_split.\
    rename(index=int, columns={"date_0": "YYYY", "date_1": "MM", "date_2": "DD"})

date_clean5 = pd.DataFrame(images_pd["SEECLICKFIX_DATE_5"].\
    str.split('T', expand=True)).rename(columns = lambda x: "part_"+str(x))
date_clean5 = date_clean5.rename(index=int, columns={"part_0": "SEECLICKFIX_DATE_5"}).\
    drop(["part_1"], axis=1)
date_5_split = pd.DataFrame(date_clean5["SEECLICKFIX_DATE_5"].\
    str.split('-', expand=True)).rename(columns = lambda x: "date_"+str(x))
date_5_split = date_5_split.\
    rename(index=int, columns={"date_0": "YYYY", "date_1": "MM", "date_2": "DD"})

print("sample of conversion:")
date_0_split.head()

sample of conversion:


,YYYY,MM,DD
0,2013,07,17
1,2013,09,18
2,2015,01,30
3,2013,09,08
4,2015,04,08


In [110]:
# collecting data on success rate of this method

matches0 = []
matches1 = []
matches2 = []
matches3 = []
matches4 = []
matches5 = []

for i in range(0,len(oak_date_split)):
    print(f"Checking incident # {i}")
    if (oak_date_split["OAK_YYYY"][i] == date_0_split["YYYY"][i]):
        if (oak_date_split["OAK_MM"][i] == date_0_split["MM"][i]):
            if (oak_date_split["OAK_DD"][i] == date_0_split["DD"][i]):
                print("-- MATCH FOUND --")
                matches0.append("MATCH")                
            elif (int(oak_date_split["OAK_DD"][i]) == int(date_0_split["DD"][i])+1):
                print("-- MATCH FOUND --")
                matches0.append("MATCH")
            elif (int(oak_date_split["OAK_DD"][i]) == int(date_0_split["DD"][i])-1):
                print("-- MATCH FOUND --")
                matches0.append("MATCH")
            else:
                matches0.append(None)
        else:
            matches0.append(None)
    else:
        matches0.append(None)
        
    if (oak_date_split["OAK_YYYY"][i] == date_1_split["YYYY"][i]):
        if (oak_date_split["OAK_MM"][i] == date_1_split["MM"][i]):
            if (oak_date_split["OAK_DD"][i] == date_1_split["DD"][i]):
                print("-- MATCH FOUND --")
                matches1.append("MATCH")                
            elif (int(oak_date_split["OAK_DD"][i]) == int(date_1_split["DD"][i])+1):
                print("-- MATCH FOUND --")
                matches1.append("MATCH")
            elif (int(oak_date_split["OAK_DD"][i]) == int(date_1_split["DD"][i])-1):
                print("-- MATCH FOUND --")
                matches1.append("MATCH")
            else:
                matches1.append(None)
        else:
            matches1.append(None)
    else:
        matches1.append(None) 
        
    if (oak_date_split["OAK_YYYY"][i] == date_2_split["YYYY"][i]):
        if (oak_date_split["OAK_MM"][i] == date_2_split["MM"][i]):
            if (oak_date_split["OAK_DD"][i] == date_2_split["DD"][i]):
                print("-- MATCH FOUND --")
                matches2.append("MATCH")                
            elif (int(oak_date_split["OAK_DD"][i]) == int(date_2_split["DD"][i])+1):
                print("-- MATCH FOUND --")
                matches2.append("MATCH")
            elif (int(oak_date_split["OAK_DD"][i]) == int(date_2_split["DD"][i])-1):
                print("-- MATCH FOUND --")
                matches2.append("MATCH")
            else:
                matches2.append(None)
        else:
            matches2.append(None)
    else:
        matches2.append(None)
        
    if (oak_date_split["OAK_YYYY"][i] == date_3_split["YYYY"][i]):
        if (oak_date_split["OAK_MM"][i] == date_3_split["MM"][i]):
            if (oak_date_split["OAK_DD"][i] == date_3_split["DD"][i]):
                print("-- MATCH FOUND --")
                matches3.append("MATCH")                
            elif (int(oak_date_split["OAK_DD"][i]) == int(date_3_split["DD"][i])+1):
                print("-- MATCH FOUND --")
                matches3.append("MATCH")
            elif (int(oak_date_split["OAK_DD"][i]) == int(date_3_split["DD"][i])-1):
                print("-- MATCH FOUND --")
                matches3.append("MATCH")
            else:
                matches3.append(None)
        else:
            matches3.append(None)
    else:
        matches3.append(None)

    if (oak_date_split["OAK_YYYY"][i] == date_4_split["YYYY"][i]):
        if (oak_date_split["OAK_MM"][i] == date_4_split["MM"][i]):
            if (oak_date_split["OAK_DD"][i] == date_4_split["DD"][i]):
                print("-- MATCH FOUND --")
                matches4.append("MATCH")                
            elif (int(oak_date_split["OAK_DD"][i]) == int(date_4_split["DD"][i])+1):
                print("-- MATCH FOUND --")
                matches4.append("MATCH")
            elif (int(oak_date_split["OAK_DD"][i]) == int(date_4_split["DD"][i])-1):
                print("-- MATCH FOUND --")
                matches4.append("MATCH")
            else:
                matches4.append(None)
        else:
            matches4.append(None)
    else:
        matches4.append(None)
        
    if (oak_date_split["OAK_YYYY"][i] == date_5_split["YYYY"][i]):
        if (oak_date_split["OAK_MM"][i] == date_5_split["MM"][i]):
            if (oak_date_split["OAK_DD"][i] == date_5_split["DD"][i]):
                print("-- MATCH FOUND --")
                matches5.append("MATCH")                
            elif (int(oak_date_split["OAK_DD"][i]) == int(date_5_split["DD"][i])+1):
                print("-- MATCH FOUND --")
                matches5.append("MATCH")
            elif (int(oak_date_split["OAK_DD"][i]) == int(date_5_split["DD"][i])-1):
                print("-- MATCH FOUND --")
                matches5.append("MATCH")
            else:
                matches5.append(None)
        else:
            matches5.append(None)
    else:
        matches5.append(None) 
    

Checking incident # 0
Checking incident # 1
Checking incident # 2
Checking incident # 3
Checking incident # 4
Checking incident # 5
Checking incident # 6
Checking incident # 7
Checking incident # 8
-- MATCH FOUND --
Checking incident # 9
Checking incident # 10
Checking incident # 11
Checking incident # 12
Checking incident # 13
-- MATCH FOUND --
Checking incident # 14
Checking incident # 15
Checking incident # 16
-- MATCH FOUND --
Checking incident # 17
-- MATCH FOUND --
-- MATCH FOUND --
Checking incident # 18
Checking incident # 19
Checking incident # 20
-- MATCH FOUND --
-- MATCH FOUND --
Checking incident # 21
Checking incident # 22
Checking incident # 23
-- MATCH FOUND --
Checking incident # 24
-- MATCH FOUND --
Checking incident # 25
Checking incident # 26
Checking incident # 27
Checking incident # 28
Checking incident # 29
-- MATCH FOUND --
Checking incident # 30
-- MATCH FOUND --
Checking incident # 31
-- MATCH FOUND --
Checking incident # 32
-- MATCH FOUND --
Checking incident

In [197]:
best_match = []
success = 0

match_date_0 = 0
match_date_1 = 0
match_date_2 = 0
match_date_3 = 0
match_date_4 = 0
match_date_5 = 0

date_results = pd.DataFrame({"DATE_0": matches0, "DATE_1": matches1, "DATE_2": matches2, "DATE_3": matches3,
    "DATE_4": matches4, "DATE_5": matches5})

# this stops after the first match
for i in range(0, len(date_results)):
    if date_results["DATE_0"][i] == "MATCH":
        best_match.append("DATE_0")
        success = success + 1
        match_date_0 = match_date_0 + 1
    elif date_results["DATE_1"][i] == "MATCH":
        best_match.append("DATE_1")
        success = success + 1
        match_date_1 = match_date_1 + 1
    elif date_results["DATE_2"][i] == "MATCH":
        best_match.append("DATE_2")
        success = success + 1
        match_date_2 = match_date_2 + 1
    elif date_results["DATE_3"][i] == "MATCH":
        best_match.append("DATE_3")
        success = success + 1
        match_date_3 = match_date_3 + 1
    elif date_results["DATE_4"][i] == "MATCH":
        best_match.append("DATE_4")
        success = success + 1
        match_date_4 = match_date_4 + 1
    elif date_results["DATE_5"][i] == "MATCH":
        best_match.append("DATE_5")
        success = success + 1
        match_date_5 = match_date_5 + 1
    else:
        best_match.append(None) 

success_0 = match_date_0 / len(best_match)
success_1 = (match_date_0 + match_date_1) / len(best_match)
success_2 = (match_date_0 + match_date_1 + match_date_2) / len(best_match) 
success_3 = (match_date_0 + match_date_1 + match_date_2 + match_date_3) / len(best_match)
success_4 = (match_date_0 + match_date_1 + match_date_2 + match_date_3 + match_date_4) / len(best_match)
success_5 = (match_date_0 + match_date_1 + match_date_2 + match_date_3 + match_date_4 + match_date_5) / len(best_match)

print(f"SUCCESS RATE - {success} out of {len(date_results)} matched!")
print("--------------------")
print("using first result:")
print("{:.2%}".format(success_0))
print("--------------------")
print("using two results:")
print("{:.2%}".format(success_1))
print("--------------------")
print("using three results:")
print("{:.2%}".format(success_2))
print("--------------------")
print("using four results:")
print("{:.2%}".format(success_3))
print("--------------------")
print("using five results:")
print("{:.2%}".format(success_4))
print("--------------------")
print("using six results:")
print("{:.2%}".format(success_5))

SUCCESS RATE - 188 out of 729 matched!
--------------------
using first result:
7.00%
--------------------
using two results:
12.07%
--------------------
using three results:
16.60%
--------------------
using four results:
21.26%
--------------------
using five results:
23.05%
--------------------
using six results:
25.79%


## Applying to Google Maps API

* With a 25.79% match rate, I proceed to tag incidents with their matches, basically separating incidents with images, applied a hover that has the description and info box with the image URL.
* LIMITATION: I would have liked to display the image but the gmaps API docs says string only. source: https://jupyter-gmaps.readthedocs.io/en/latest/api.html

In [133]:
best_image = []
best_description = []

images_pd["INCIDENT_MATCH"] = best_match

for i in range(0, len(images_pd)):
    if images_pd["INCIDENT_MATCH"][i] == "DATE_0":
        best_image.append(images_pd["IMAGE_URL_0"][i])
        best_description.append(images_pd["DESCRIPTION_0"][i])
        
    elif images_pd["INCIDENT_MATCH"][i] == "DATE_1":
        best_image.append(images_pd["IMAGE_URL_1"][i])
        best_description.append(images_pd["DESCRIPTION_1"][i])
        
    elif images_pd["INCIDENT_MATCH"][i] == "DATE_2":
        best_image.append(images_pd["IMAGE_URL_2"][i])
        best_description.append(images_pd["DESCRIPTION_2"][i])
        
    elif images_pd["INCIDENT_MATCH"][i] == "DATE_3":
        best_image.append(images_pd["IMAGE_URL_3"][i])
        best_description.append(images_pd["DESCRIPTION_3"][i])
        
    elif images_pd["INCIDENT_MATCH"][i] == "DATE_4":
        best_image.append(images_pd["IMAGE_URL_4"][i])
        best_description.append(images_pd["DESCRIPTION_4"][i])  
    
    elif images_pd["INCIDENT_MATCH"][i] == "DATE_5":
        best_image.append(images_pd["IMAGE_URL_5"][i])
        best_description.append(images_pd["DESCRIPTION_5"][i]) 
        
    else:
        best_image.append(None)
        best_description.append(None)  

images_pd["BEST_IMAGE"] = best_image
images_pd["BEST_DESCRIPTION"] = best_description

images_pd.head(30)

,IMAGE_URL_0,DESCRIPTION_0,SEECLICKFIX_DATE_0,IMAGE_URL_1,DESCRIPTION_1,SEECLICKFIX_DATE_1,IMAGE_URL_2,DESCRIPTION_2,SEECLICKFIX_DATE_2,IMAGE_URL_3,...,SEECLICKFIX_DATE_4,IMAGE_URL_5,DESCRIPTION_5,SEECLICKFIX_DATE_5,OAK_311_DATE,LATITUDE,LONGITUDE,INCIDENT_MATCH,BEST_IMAGE,BEST_DESCRIPTION
0,https://seeclickfix.com/files/issue_images/001...,Overgrown and blocking side walk.,2013-07-17T22:51:11-04:00,None,"Many years ago, the City of Oakland staff plan...",2016-01-29T12:34:56-05:00,None,None,None,None,...,None,None,None,None,11/04/2017 03:30:20 PM,37.803094,-122.242301,None,None,None
1,https://seeclickfix.com/files/issue_images/001...,Trees on public sidewalk need to be trimmed. ...,2013-09-18T14:55:11-04:00,https://seeclickfix.com/files/issue_images/001...,Treedown at fourth and Alice,2014-03-20T10:10:48-04:00,https://seeclickfix.com/files/issue_images/003...,Huge branch fallen down in street,2015-09-14T17:11:33-04:00,https://seeclickfix.com/files/issue_images/004...,...,2015-09-04T12:53:07-04:00,https://seeclickfix.com/files/issue_images/004...,Branch broken,2015-11-12T19:21:18-05:00,11/06/2017 02:08:59 PM,37.794393,-122.271948,None,None,None
2,None,Branches cut by city during Dec storms have be...,2015-01-30T17:04:12-05:00,None,None,None,None,None,None,None,...,None,None,None,None,05/18/2018 03:20:47 PM,37.783582,-122.141797,None,None,None
3,https://seeclickfix.com/files/issue_images/001...,This tree needs to be replaced!,2013-09-08T12:15:58-04:00,https://seeclickfix.com/files/issue_images/002...,The residence located at 2247 E. 15th Street h...,2015-02-08T19:01:39-05:00,None,None,None,None,...,None,None,None,None,11/08/2018 02:29:43 PM,37.785719,-122.234163,None,None,None
4,https://seeclickfix.com/files/issue_images/003...,need some maintainance.,2015-04-08T20:44:23-04:00,None,hazardous tree at the residence: 3232 Elmwood...,2016-11-04T12:32:16-04:00,https://seeclickfix.com/files/issue_images/002...,Large fallen tree branch (on my car - I was ab...,2014-08-13T23:18:33-04:00,https://seeclickfix.com/files/issue_images/002...,...,2016-12-22T17:28:00-05:00,None,None,None,10/14/2017 02:21:30 PM,37.772056,-122.229826,None,None,None
5,https://seeclickfix.com/files/issue_images/001...,Please come trim these trees they are too tall...,2013-11-23T10:57:46-05:00,https://seeclickfix.com/files/issue_images/001...,Tree uprooted looks like it was hit by a car. ...,2014-02-23T14:03:37-05:00,https://seeclickfix.com/files/issue_images/001...,Please come trim the trees in the creek tree f...,2013-11-23T10:59:48-05:00,None,...,2015-06-22T14:01:52-04:00,https://seeclickfix.com/files/issue_images/008...,The tree canopy is too low. It has attracted ...,2017-09-25T22:12:12-04:00,11/26/2018 05:18:44 PM,37.786512,-122.225857,None,None,None
6,None,"On MacArthur in the westbound direction, a tre...",2015-04-08T03:27:43-04:00,https://seeclickfix.com/files/issue_images/002...,No one has trimmed dis @#$% of tree yet! No on...,2014-11-05T12:21:40-05:00,https://seeclickfix.com/files/issue_images/002...,Untrimmed bottom foliage blocking driver's vi...,2014-07-08T02:16:59-04:00,https://seeclickfix.com/files/issue_images/008...,...,None,None,None,None,04/26/2013 08:21:25 AM,37.772221,-122.173052,None,None,None
7,https://seeclickfix.com/files/issue_images/002...,The landscaping along the East side of Pleasan...,2014-09-02T19:03:44-04:00,None,None,None,None,None,None,None,...,None,None,None,None,06/08/2014 12:30:16 AM,37.831536,-122.248894,None,None,None
8,https://seeclickfix.com/files/issue_images/003...,downed tree from car crash,2015-03-20T18:57:38-04:00,None,Tree down blocking rt lane east bound Keller ave,2014-05-02T19:10:57-04:00,None,downed trees in median strip\r\nthis has occur...,2014-07-25T16:05:27-04:00,None,...,None,None,None,None,07/25/2014 01:10:15 PM,37.772736,-122.143045,DATE_2,None,downed trees in median strip\r\nthis has occur...
9,https://seeclickfix.com/files/issue_images/003...,Possible campfire in woods on south side 580 a...,2015-05-12T1

In [126]:
images_pd.head(5)

,IMAGE_URL_0,DESCRIPTION_0,SEECLICKFIX_DATE_0,IMAGE_URL_1,DESCRIPTION_1,SEECLICKFIX_DATE_1,IMAGE_URL_2,DESCRIPTION_2,SEECLICKFIX_DATE_2,IMAGE_URL_3,...,IMAGE_URL_4,DESCRIPTION_4,SEECLICKFIX_DATE_4,IMAGE_URL_5,DESCRIPTION_5,SEECLICKFIX_DATE_5,OAK_311_DATE,LATITUDE,LONGITUDE,INCIDENT_MATCH
0,https://seeclickfix.com/files/issue_images/001...,Overgrown and blocking side walk.,2013-07-17T22:51:11-04:00,None,"Many years ago, the City of Oakland staff plan...",2016-01-29T12:34:56-05:00,None,None,None,None,...,None,None,None,None,None,None,11/04/2017 03:30:20 PM,37.803094,-122.242301,None
1,https://seeclickfix.com/files/issue_images/001...,Trees on public sidewalk need to be trimmed. ...,2013-09-18T14:55:11-04:00,https://seeclickfix.com/files/issue_images/001...,Treedown at fourth and Alice,2014-03-20T10:10:48-04:00,https://seeclickfix.com/files/issue_images/003...,Huge branch fallen down in street,2015-09-14T17:11:33-04:00,https://seeclickfix.com/files/issue_images/004...,...,https://seeclickfix.com/files/issue_images/003...,Huge branch on sidewalk,2015-09-04T12:53:07-04:00,https://seeclickfix.com/files/issue_images/004...,Branch broken,2015-11-12T19:21:18-05:00,11/06/2017 02:08:59 PM,37.794393,-122.271948,None
2,None,Branches cut by city during Dec storms have be...,2015-01-30T17:04:12-05:00,None,None,None,None,None,None,None,...,None,None,None,None,None,None,05/18/2018 03:20:47 PM,37.783582,-122.141797,None
3,https://seeclickfix.com/files/issue_images/001...,This tree needs to be replaced!,2013-09-08T12:15:58-04:00,https://seeclickfix.com/files/issue_images/002...,The residence located at 2247 E. 15th Street h...,2015-02-08T19:01:39-05:00,None,None,None,None,...,None,None,None,None,None,None,11/08/2018 02:29:43 PM,37.785719,-122.234163,None
4,https://seeclickfix.com/files/issue_images/003...,need some maintainance.,2015-04-08T20:44:23-04:00,None,hazardous tree at the residence: 3232 Elmwood...,2016-11-04T12:32:16-04:00,https://seeclickfix.com/files/issue_images/002...,Large fallen tree branch (on my car - I was ab...,2014-08-13T23:18:33-04:00,https://seeclickfix.com/files/issue_images/002...,...,None,The palm tree in front of 614 Lancaster is ful...,2016-12-22T17:28:00-05:00,None,None,None,10/14/2017 02:21:30 PM,37.772056,-122.229826,None


In [153]:
# isolating results with no images to keep separate

no_image = pd.DataFrame(images_pd[images_pd["BEST_IMAGE"].isnull()])
no_image = no_image.reset_index(drop=True)

no_image_coord = pd.DataFrame({"LATITUDE": no_image["LATITUDE"], "LONGITUDE": no_image["LONGITUDE"]})

no_image_coord.to_csv("final_no_images_coordinates.csv", index=True, header=True)
no_image_coord.head()

,LATITUDE,LONGITUDE
0,37.803094,-122.242301
1,37.794393,-122.271948
2,37.783582,-122.141797
3,37.785719,-122.234163
4,37.772056,-122.229826


In [155]:
# results with images

has_img = pd.DataFrame(images_pd[images_pd["BEST_IMAGE"].notnull()])
has_img = has_img.reset_index(drop=True)

img_coord = pd.DataFrame({"LATITUDE": has_img["LATITUDE"], "LONGITUDE": has_img["LONGITUDE"]})
img_links = pd.DataFrame({"URL": has_img["BEST_IMAGE"], "DESCRIPTION": has_img["BEST_DESCRIPTION"]})

# these are 2 separate CSV for gmaps because coordinates are a pair

img_coord.to_csv("final_images_coordinates.csv", index=True, header=True)
img_links.to_csv("final_images_links.csv", index=True, header=True)

img_coord.head()

,LATITUDE,LONGITUDE
0,37.769628,-122.175942
1,37.791221,-122.243675
2,37.799699,-122.248284
3,37.811813,-122.303050
4,37.807255,-122.241624


In [144]:
img_links.head(15)

,URL,DESCRIPTION
0,https://seeclickfix.com/files/issue_images/002...,No one has trimmed dis @#$% of tree yet! No on...
1,https://seeclickfix.com/files/issue_images/003...,"Tree brace broke off, tree is now limping to t..."
2,https://seeclickfix.com/files/issue_images/003...,Please keep a eye on the redwood tree is dryin...
3,https://seeclickfix.com/files/issue_images/004...,Dead tree at 10th and frontage.
4,https://seeclickfix.com/files/issue_images/006...,Tree blocking pedestrian walkway on Lake Park ...
5,https://seeclickfix.com/files/issue_images/006...,I will also call. Tree fell and hangs over roa...
6,https://seeclickfix.com/files/issue_images/007...,"I don't think this is an emergency, but a prev..."
7,https://seeclickfix.com/files/issue_images/007...,DANGEROUS TREE dead and leaning at 70-degree a...
8,https://seeclickfix.com/files/issue_images/007...,Trees on union between 12 and 14 are awful. Y...
9,https://seeclickfix.com/files/issue_images/008...,Trees need to be trimmed


In [198]:
# this will be the final table for viewing purposes

img_html = []

img_df = pd.DataFrame({"REQUEST_DATE": has_img["OAK_311_DATE"], "SOURCE_URL": has_img["BEST_IMAGE"],
    "DESCRIPTION": has_img["BEST_DESCRIPTION"],"LATITUDE": has_img["LATITUDE"], "LONGITUDE": has_img["LONGITUDE"]})

img_df.to_csv("final_results_table.csv", index=True, header=True)
img_df.head()

,REQUEST_DATE,SOURCE_URL,DESCRIPTION,LATITUDE,LONGITUDE
0,11/05/2014 09:30:27 AM,https://seeclickfix.com/files/issue_images/002...,No one has trimmed dis @#$% of tree yet! No on...,37.769628,-122.175942
1,06/04/2015 04:50:18 PM,https://seeclickfix.com/files/issue_images/003...,"Tree brace broke off, tree is now limping to t...",37.791221,-122.243675
2,08/03/2015 01:00:26 PM,https://seeclickfix.com/files/issue_images/003...,Please keep a eye on the redwood tree is dryin...,37.799699,-122.248284
3,02/19/2016 11:30:52 AM,https://seeclickfix.com/files/issue_images/004...,Dead tree at 10th and frontage.,37.811813,-122.303050
4,02/11/2017 04:58:11 PM,https://seeclickfix.com/files/issue_images/006...,Tree blocking pedestrian walkway on Lake Park ...,37.807255,-122.241624


In [143]:
# google maps API

# REDACTED API KEY - please use your own, thank you!

gmaps.configure(api_key="")

fig = gmaps.figure()

symbol = gmaps.symbol_layer(no_image_coord, fill_color='blue')
img_marker = gmaps.marker_layer(img_coord, info_box_content=img_links["URL"], hover_text=img_links["DESCRIPTION"])

fig.add_layer(symbol)
fig.add_layer(img_marker)

fig

Figure(layout=FigureLayout(height='420px'))

## FINAL REPORT

* Please see 'map_screenshot.png' in the repository for an example of clicking on the map.  You will see a still-dangerous tree at that location, with images and description so you can keep an eye out!  This shows different results from only "open" reports as there are other statuses that haven't resulted in someone correcting the issue (such as lack of funding).

### EXTRACT / TRANSFORM / LOAD

* Sources of data: (links and instructions throughout the jupyter notebook) City of Oakland Service Requests, SeeClickFix, EPSG.io Coordinate Systems Worldwide, & Google Maps API


* SeeClickFix is an app meant for people to report hazards and other incidents to be addressed or resolved, usually by the local government.  The location data for reports tends to be very accurate because the submitter pins the location on a map. When searching for specific hazards, a search query in the app for a very basic word like 'tree' brings up a lot of false positives, basically any word that contains those letter in it, such as 'street' (if you search for the word tree in quotation marks, you get zero results).  There is no way to filter by fixed categories either unless you know the exact name of the category to type in.  They have an API that requires either coordinates or account ID but can pull up responses that are only partial matches (eg correct or nearby coordinates but not matching 'service_name' or 'requested_datetime' inputs).  URL: https://seeclickfix.com/open311/v2/docs


* The City of Oakland provides a large ongoing record of service requests.  One source of such reports is from SeeClickFix, but the city maintains different metadata (such as converting location from latitude/longitude to NAD 1983 StatePlane California III FIPS 0403 Feet and dropping some metadata in SeeClickFix such as description and the user-submitted photo).  However, their dataset (a CSV w/ >500k rows) is easier to filter/sort by different columns (such as by municipal district or request category) to acquire data which can then be plugged back into the SeeClickFix API.  Unfortunately the request ID reference numbers cannot be cross-referenced (you can find the reference number from Oakland in the SeeClickFix under comments as well as a field in the app called 'Remote Id' which cannot be searched in the SeeClickFix API).  The data can be access with a JSON query but times out due to the size of the dataset.  URL: https://data.oaklandnet.com/Infrastructure/Service-requests-received-by-the-Oakland-Call-Cent/quth-gb8e


* Types of transforming: cleaning, sorting, filtering (by SeeClickFix, by Tree incidents), splitting (date info for comparison purposes), converting (coordinates)

* Final production database: load coordinates & other metadata (CSV format) into Google Maps API using parameters described here: https://jupyter-gmaps.readthedocs.io/en/latest/api.html